In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install 'keras-applications'
!pip install 'efficientnet'

     |████████████████████████████████| 50 kB 730 kB/s 


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras import models, Sequential, Input, layers, optimizers, regularizers, callbacks, losses, metrics 
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold

import efficientnet.tfkeras as effnet
from tensorflow.keras.layers import Input
import warnings
warnings.filterwarnings("ignore")
from catboost import CatBoostRegressor
from xgboost.sklearn import XGBClassifier, XGBRegressor
%matplotlib inline

In [4]:
# FOR KAGGLE
train_images_dir = '../input/petfinder-pawpularity-score/train/'
test_images_dir = '../input/petfinder-pawpularity-score/test/'

# FOR LOCAL PC
# train_images_dir = './dataset/train/'
# test_images_dir = './dataset/test/'

In [5]:
train_df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
train_df.loc[:, 'filename'] = train_images_dir + train_df['Id'] + '.jpg'

test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df.loc[:, 'filename'] = test_images_dir + test_df['Id'] + '.jpg'


Q = 30
# Set a specific label to be able to perform stratification
train_df['stratify_label'] = pd.qcut(train_df['Pawpularity'], q = Q, labels = range(Q))
# Label value to be used for feature model 'classification' training.
train_df['target_value'] = train_df['Pawpularity'] / 100.

In [6]:
img_size = (320, 320, 3)
BATCH_SIZE = 32
EPOCHS = 10
LEARNING_RATE = 0.000005
FEATURE_FOLDS = 10
FOLDS = 6
SEED = 42

In [7]:
def augment_image(image, label=None):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_saturation(image, 0.95, 1.05)
    image = tf.image.random_brightness(image, 0.05)
    image = tf.image.random_contrast(image, 0.95, 1.05)
    image = tf.image.random_hue(image, 0.05)
    return image, label

def parse_image(filename, label=None):
    image_string = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image_string, channels=img_size[2])
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, img_size[:2])
    return image, label

def get_dataset(filenames, label=None, shuffle=False, augment=False, repeat=False):
    dataset = tf.data.TFRecordDataset.from_tensor_slices((filenames, label))     
    dataset = dataset.map(parse_image, num_parallel_calls=4)
    if augment:
        dataset = dataset.map(augment_image, num_parallel_calls=4)
    if repeat:
        dataset = dataset.repeat()
    if shuffle:
        dataset = dataset.shuffle(1024, reshuffle_each_iteration = True)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(True)
    return dataset


In [8]:
def build_model(): 
    # Create and Compile Model and show Summary
    model = effnet.EfficientNetB2(include_top = False, 
                                  classes = None, 
                                  input_shape = img_size, 
                                  weights = 'noisy-student', 
                                  pooling = 'avg')

    # Set all layers to Trainable except BN layers
    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = False
        else:
            layer.trainable = True

    
    X = tf.keras.layers.Dropout(0.25)(model.output)
    output = tf.keras.layers.Dense(1, activation = 'sigmoid')(X)
    model = tf.keras.Model(inputs = model.input, outputs = output)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE), 
                  loss = tf.keras.losses.BinaryCrossentropy(), 
                  metrics = [tf.keras.metrics.RootMeanSquaredError('rmse')])        
    
    return model   

In [9]:
preds_final = np.zeros((test_df.shape[0], 1))
for fold_index in range(FEATURE_FOLDS):
    tf.keras.backend.clear_session()

    model = build_model()
    model.load_weights(f'../input/effnet/feature_model_{fold_index}.h5')
    model = tf.keras.Model(inputs = model.input, outputs = model.layers[-3].output)    
    
    # Feature Extraction
    train_dataset = get_dataset(train_df['filename'], train_df['target_value'])
    test_dataset = get_dataset(test_df['filename'])
    train_features = model.predict(train_dataset)
    test_features = model.predict(test_dataset)

    kfold = StratifiedKFold(n_splits = FOLDS, shuffle = True, random_state = SEED)
    for train, val in kfold.split(train_features, train_df['stratify_label']):
        X_train, y_train = train_features[train,:], train_df.loc[train,'Pawpularity']
        X_val, y_val = train_features[val,:], train_df.loc[val,'Pawpularity']
        model = XGBRegressor(learning_rate=0.07, 
                             gamma=0.2, 
                             max_depth=5, 
                             n_estimators=100, 
                             reg_alpha=25.6, 
                             reg_lambda=0.8, 
                             random_state=SEED,
                             verbosity=0)
        model.fit(X_train, y_train, 
                  eval_set=[(X_val, y_val)], 
                  early_stopping_rounds = 100, 
                  eval_metric='rmse')
        preds_final += np.array([model.predict(test_features)]).T
        print(f'fold complete {SEED-42}')

        SEED += 1


2021-12-20 06:16:44.303326: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-20 06:16:44.400234: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-20 06:16:44.400962: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-12-20 06:16:44.402095: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

31793152/31782304 [==============================] - 2s 0us/step


2021-12-20 06:16:53.209628: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-12-20 06:16:55.993329: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


[0]	validation_0-rmse:40.60831
[1]	validation_0-rmse:38.53646
[2]	validation_0-rmse:36.64605
[3]	validation_0-rmse:34.92480
[4]	validation_0-rmse:33.36474
[5]	validation_0-rmse:31.94971
[6]	validation_0-rmse:30.68101
[7]	validation_0-rmse:29.53302
[8]	validation_0-rmse:28.50365
[9]	validation_0-rmse:27.59117
[10]	validation_0-rmse:26.77232
[11]	validation_0-rmse:26.03332
[12]	validation_0-rmse:25.38139
[13]	validation_0-rmse:24.79945
[14]	validation_0-rmse:24.28537
[15]	validation_0-rmse:23.84106
[16]	validation_0-rmse:23.43307
[17]	validation_0-rmse:23.07015
[18]	validation_0-rmse:22.76074
[19]	validation_0-rmse:22.49318
[20]	validation_0-rmse:22.25180
[21]	validation_0-rmse:22.04727
[22]	validation_0-rmse:21.87238
[23]	validation_0-rmse:21.71604
[24]	validation_0-rmse:21.57910
[25]	validation_0-rmse:21.45323
[26]	validation_0-rmse:21.36194
[27]	validation_0-rmse:21.27702
[28]	validation_0-rmse:21.20425
[29]	validation_0-rmse:21.13312
[30]	validation_0-rmse:21.08313
[31]	validation_0-

In [10]:
preds_final /= (FOLDS*FEATURE_FOLDS)
submisson_df = test_df[['Id']]
submisson_df['Pawpularity'] = preds_final

In [11]:
submisson_df.to_csv('submission.csv', index=False)